In [1]:
import os
os.chdir('../')
import model.op
from model.op import FleetSizeOptimizer
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
kappa = np.array([[0,82.31,57.42],[65.78,0,43.78], [43.78,57.42,0]])/160*100
tau = np.array([[0, 19, 11.45], [19.7, 0, 13.3], [13.3,11.45, 0]])

# kappa = kappa[0:2,0:2]
# tau = tau[0:2,0:2]
op = FleetSizeOptimizer(flight_time=tau, energy_consumption=kappa, schedule='schedule_5min_0612.csv', fixed_cost=10)

In [3]:
op.optimize('HHR_WARNER/HRR_and_middle_point.txt', charging_station=[True, False, True])

Set parameter Username
Academic license - for non-commercial use only - expires 2024-04-29
Set parameter MIPGap to value 0.05
Set parameter FeasibilityTol to value 1e-07
Gurobi Optimizer version 10.0.3 build v10.0.3rc0 (mac64[arm])

CPU model: Apple M2 Pro
Thread count: 10 physical cores, 10 logical processors, using up to 10 threads

Optimize a model with 34684 rows, 551133 columns and 1291481 nonzeros
Model fingerprint: 0xf1000437
Variable types: 0 continuous, 551133 integer (0 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e-07, 1e+01]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 5e+00]
Presolve removed 14646 rows and 198026 columns
Presolve time: 0.92s
Presolved: 20038 rows, 353107 columns, 703742 nonzeros
Variable types: 0 continuous, 353107 integer (12 binary)
Deterministic concurrent LP optimizer: primal simplex, dual simplex, and barrier
Showing barrier log only...

Root barrier log...

Ordering time: 0.72s

Barrier statis

In [4]:
op.parse_result('HHR_WARNER/HRR_and_middle_point.txt')
op.calculate_aircraft_states()
# op.plot_aircraft_state()

In [6]:
dist = np.array([[0, 30, 15], [30, 0, 15], [15, 15, 0]])
# dist = dist[0:2,0:2]
op.get_summary_statistics(dist)

Fleet size: 134.0
Total number of pads: 179; [68 30 81] 
Total number of flights: 2283.0; demand: 771.0; repositioning: 1512.0
Total energy consumption: 70274.0 kWh
Total aircraft miles: 92400.0 mi
Ratio of revenue aircraft miles to aircraft miles: 0.375487012987013


# Wind Variation

In [18]:
from tqdm import tqdm

In [13]:
hhr_warner_energy = pd.read_csv('input/route_profile/hhr_warner_energy.csv').to_numpy()[:,1:]
warner_hhr_energy = pd.read_csv('input/route_profile/warner_hhr_energy.csv').to_numpy()[:,1:]
warner_hhr_flight_duration = pd.read_csv('input/route_profile/warner_hhr_flight_duration.csv').to_numpy()[:,1:]
hhr_warner_flight_duration = pd.read_csv('input/route_profile/hhr_warner_flight_duration.csv').to_numpy()[:,1:]

In [19]:
mag = np.arange(0, 60, 10)
angel = np.arange(0, 365+45, 45)
dist = np.array([[0,30],[30,0]])
fleetsize_all = np.zeros((hhr_warner_energy.shape[0], hhr_warner_energy.shape[1]))
num_pads_all = np.zeros((hhr_warner_energy.shape[0], hhr_warner_energy.shape[1]))
vertiport_pads_all = np.zeros((hhr_warner_energy.shape[0], hhr_warner_energy.shape[1], 2))
num_flight_all = np.zeros((hhr_warner_energy.shape[0], hhr_warner_energy.shape[1]))
energy_consumption_all = np.zeros((hhr_warner_energy.shape[0], hhr_warner_energy.shape[1]))
aircraft_miles_all = np.zeros((hhr_warner_energy.shape[0], hhr_warner_energy.shape[1]))
revenue_miles_all = np.zeros((hhr_warner_energy.shape[0], hhr_warner_energy.shape[1]))


for i in tqdm(range(hhr_warner_energy.shape[0])):
    for j in range(hhr_warner_energy.shape[1]):
        kappa = np.array([[0, hhr_warner_energy[i,j]], [warner_hhr_energy[i,j], 0]])/160*100
        tau = np.array([[0, hhr_warner_flight_duration[i,j]], [warner_hhr_flight_duration[i,j], 0]])
        op = FleetSizeOptimizer(flight_time=tau, energy_consumption=kappa, schedule='schedule_5min_0612.csv')
        op.optimize(f'HHR_WARNER/wind_variation/ang_{angel[i]}_mag_{mag[j]}.txt', charging_station=[True, True])
        op.parse_result(f'HHR_WARNER/wind_variation/ang_{angel[i]}_mag_{mag[j]}.txt')
        op.calculate_aircraft_states()

        fleetsize, num_pads, vertiport_pads, num_flight, energy_consumption, aircraft_miles, revenue_miles = op.get_summary_statistics(dist, True)

        fleetsize_all[i,j] = fleetsize
        num_pads_all[i,j] = num_pads
        vertiport_pads_all[i,j,:] = vertiport_pads
        num_flight_all[i,j] = num_flight
        energy_consumption_all[i,j] = energy_consumption
        aircraft_miles_all[i,j] = aircraft_miles
        revenue_miles_all[i,j] = revenue_miles




  0%|          | 0/6 [00:00<?, ?it/s]

Set parameter MIPGap to value 0.05
Set parameter FeasibilityTol to value 1e-07
Gurobi Optimizer version 10.0.3 build v10.0.3rc0 (mac64[arm])

CPU model: Apple M2 Pro
Thread count: 10 physical cores, 10 logical processors, using up to 10 threads

Optimize a model with 21700 rows, 349272 columns and 705872 nonzeros
Model fingerprint: 0x2c5697e1
Variable types: 0 continuous, 349272 integer (0 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e-07, 1e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 5e+00]
Presolve removed 1938 rows and 7487 columns
Presolve time: 0.56s
Presolved: 19762 rows, 341785 columns, 692513 nonzeros
Variable types: 0 continuous, 341785 integer (0 binary)
Deterministic concurrent LP optimizer: primal simplex, dual simplex, and barrier
Showing barrier log only...

Root barrier log...

Ordering time: 0.56s

Barrier statistics:
 AA' NZ     : 3.608e+05
 Factor NZ  : 8.409e+06 (roughly 200 MB of memory)
 Factor Ops : 5